In [1]:
import cv2
import numpy as np
import os
from selenium import webdriver
from PIL import Image
from datetime import datetime
import pandas as pd
import sqlalchemy

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [3]:
info_dict = {'xy_coord': [], 'date': [], 'timestamp': [], 'coord_stamp': []}

In [ ]:
session_dict = {'xy_coord': [], 'date': [], 'timestamp': [], 'coord_stamp': []}

In [5]:
region_dict = {'Центр': (255, 255, 255), 'Юго-Запад': (0, 38, 255), 'Фили': (182, 255, 0), 'Строгино': (255, 106, 0), 'Тушино': (178, 0, 255), 'Ховрино': (0, 127, 127),
            'Дмитровка': (38, 127, 0), 'Останкино': (128, 128, 128), 'Медведково': (0, 255, 255),'Лефортово': (127, 0, 55), 'Измайлово': (127, 51, 0), 'Перово': (255, 216, 0),
            'Вешняки': (178, 255, 193), 'Кузьминки': (124, 94, 124), 'Люблино': (0, 148, 255), 'Зябликово': (127, 0, 0), 'Каширка': (56, 54, 63), 'Сукино Болото': (0, 19, 127)}

In [6]:
def do_reg(xy_coord):
    if img_color_dist.getpixel((xy_coord[0],xy_coord[1]))[:3] in region_dict.values():
        return list(region_dict.keys())[list(region_dict.values()).index(img_color_dist.getpixel((xy_coord[0],xy_coord[1]))[:3])]
    else:
        return 'За МКАД'

In [23]:
engine = sqlalchemy.create_engine('mysql+mysqlconnector://root:@localhost:3306/accidents')
def send_values(df):

    # transform column values to string for MySQL database
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(x) if isinstance(x, tuple) else x)

    con = engine.connect()
    table_name = 'accidents'
    df.to_sql(table_name, con, if_exists='append', index=False)
    con.close()

In [21]:
# https://chromedriver.chromium.org/home
chromedriver = "chromedriver-win64/chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument("--start-maximized")

service = Service(executable_path=chromedriver)
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()
driver.get("https://yandex.ru/maps/213/moscow/?l=trf%2Ctrfe&ll=37.838392%2C55.739488&z=11.16")
driver.get_screenshot_as_file("screenshot.png")
driver.quit()

In [22]:
img = Image.open(os.path.join(os.getcwd(), "screenshot.png"))
area = (450, 50, 1200, 923)
img.crop(area).save(os.path.join(os.getcwd(), "screenshot_crop.png"))
img_color_dist = Image.open(os.path.join(os.getcwd(), "region_dist_color.png"))
image = cv2.imread(os.path.join(os.getcwd(), "screenshot_crop.png"), cv2.IMREAD_COLOR)

template = cv2.imread(os.path.join(os.getcwd(), "scr_temp.png"), cv2.IMREAD_COLOR)
h, w = template.shape[:2]

method = cv2.TM_CCOEFF_NORMED

threshold = 0.8

res = cv2.matchTemplate(image, template, method)


# fake out max_val for first run through loop
max_val = 1
x_c = -10000
y_c = -10000

len_ses_dict = 0

while max_val > threshold:
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    if x_c == (2*max_loc[0]+w)/2 and y_c == (2*max_loc[1]+h)/2:
        break
    else:
        res[max_loc[1]-h//2:max_loc[1]+h//2+1, max_loc[0]-w//2:max_loc[0]+w//2+1] = 0   
        image = cv2.rectangle(image,(max_loc[0],max_loc[1]), (max_loc[0]+w+1, max_loc[1]+h+1), (255,0,0) )
        x_c = (2*max_loc[0]+w)/2
        y_c = (2*max_loc[1]+h)/2
        if (x_c, y_c) not in ['xy_coord']:
            info_dict['xy_coord'].append( (x_c, y_c))
            info_dict['date'].append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            info_dict['timestamp'].append(int(datetime.now().timestamp()))
            info_dict['coord_stamp'].append((x_c, y_c, int(datetime.now().timestamp())))
            
            session_dict['xy_coord'].append( (x_c, y_c))
            session_dict['date'].append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            session_dict['timestamp'].append(int(datetime.now().timestamp()))
            session_dict['coord_stamp'].append((x_c, y_c, int(datetime.now().timestamp())))
        elif (x_c, y_c) in info_dict['xy_coord'] and int(datetime.now().timestamp()) - [z for x, y, z in info_dict['coord_stamp'] if x == x_c and y == y_c][0] > 5000:
            info_dict['xy_coord'].append(x_c, y_c)
            info_dict['date'].append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            info_dict['timestamp'].append(int(datetime.now().timestamp()))
            info_dict['coord_stamp'].append((x_c, y_c, int(datetime.now().timestamp())))
            
            session_dict['xy_coord'].append( (x_c, y_c))
            session_dict['date'].append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            session_dict['timestamp'].append(int(datetime.now().timestamp()))
            session_dict['coord_stamp'].append((x_c, y_c, int(datetime.now().timestamp())))
            
cv2.imwrite(r'output.png', image)

if  len_ses_dict != len(session_dict['xy_coord']):
    len_ses_dict = len(session_dict['xy_coord'])
    df_session = pd.DataFrame.from_dict(session_dict, orient='index')
    df_session = df_session.transpose()
    df_session['region'] = df_session.apply(lambda x: do_reg(x['xy_coord']), axis =1)
    send_values(df_session)
    session_dict = {'xy_coord': [], 'date': [], 'timestamp': [], 'coord_stamp': []}
    df_session = pd.DataFrame()
    
img_color_dist.close()
img.close()